## Import and clean weather dataset
Central Park Monthly Avg Temp and Precipitation from 1869 - 2018:
- https://www.weather.gov/media/okx/Climate/CentralPark/monthlyannualtemp.pdf
- https://www.weather.gov/media/okx/Climate/CentralPark/monthlyannualprecip.pdf

In [10]:
import pandas as pd
import datetime as dt

In [2]:
# Function to convert PDF tables into CSVs using https://pdftables.com/
# Only get 50 free calls to the API! 
def pdfToTable(PDFfilename, apiKey, fileExt, downloadDir):

    fileData = (PDFfilename, open(PDFfilename, 'rb')) #"rb" stands for "read bytes"
    files = {'f': fileData} 
    postUrl = "https://pdftables.com/api?key={0}&format={1}".format(apiKey, fileExt)

    response = requests.post(postUrl, files=files)
    response.raise_for_status() # ensure we notice bad responses

    with open(downloadDir, "wb") as f:
        f.write(response.content) #write data to csv

#### Monthly avg temp data

In [5]:
# import monthly temp data
# don't run this over and over again - will run out of API credits
#pdfToTable('centralpark_monthlyannualtemp.pdf', 'fehzjlvcj5ky', 'csv', 'centralpark_monthlyannualtemp.csv')
temp = pd.read_csv('centralpark_monthlyannualtemp.csv', header=2)
temp.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL
0,1869,35.1,34.5,34.8,49.2,57.7,69.3,72.8,71.8,65.6,50.9,40.3,34.7,51.4
1,1870,37.5,31.3,34.1,50.7,60.9,72.9,76.6,75.3,67.6,56.7,45.5,34.1,53.6
2,1871,28.3,30.2,44.2,52.0,60.4,68.2,72.3,73.6,60.8,55.6,38.8,29.2,51.1
3,1872,28.8,29.9,30.5,49.4,61.5,71.2,77.5,75.6,66.4,53.2,41.0,26.7,51.0
4,1873,28.6,29.5,35.7,46.7,58.8,70.3,75.4,72.0,65.4,55.8,37.0,36.5,51.0


In [6]:
# clean up:
# remove rows with repeated column names from original PDF
temp = temp[temp['YEAR']!='YEAR'] 

# convert YEAR to int
temp['YEAR'] = temp['YEAR'].astype(int) 

# keep relevant years
temp = temp[(temp.YEAR>=1928) & (temp.YEAR<=1972)] 

# convert month values to floats
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
          'JUL', 'AUG', 'SEP','OCT', 'NOV', 'DEC']

temp[months] = temp[months].astype(float)

# drop ANNUAL column
temp.drop('ANNUAL', axis=1, inplace=True)

temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 60 to 105
Data columns (total 13 columns):
YEAR    45 non-null int64
JAN     45 non-null float64
FEB     45 non-null float64
MAR     45 non-null float64
APR     45 non-null float64
MAY     45 non-null float64
JUN     45 non-null float64
JUL     45 non-null float64
AUG     45 non-null float64
SEP     45 non-null float64
OCT     45 non-null float64
NOV     45 non-null float64
DEC     45 non-null float64
dtypes: float64(12), int64(1)
memory usage: 4.9 KB


In [7]:
temp.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
60,1928,34.0,33.0,38.8,48.4,59.4,67.4,75.9,75.4,64.7,58.9,47.4,39.3
61,1929,31.9,32.9,45.1,51.0,61.7,71.5,75.9,73.1,69.4,54.6,46.2,35.8
62,1930,33.3,37.0,40.3,48.4,63.7,73.5,76.4,73.6,72.0,55.6,45.5,34.6
63,1931,32.8,33.9,40.3,51.1,61.5,70.7,77.5,75.2,72.3,61.3,51.9,41.0
64,1932,43.2,36.1,37.3,49.0,62.0,70.5,76.1,76.7,69.5,58.5,43.9,39.1


In [8]:
# reshape the data set to match our series
temp_list = []

for _, row in temp.iterrows():
    for month in range(1,13):
        temp_list.append(row[month])
    
temp_list[:5]

[34.0, 33.0, 38.799999999999997, 48.399999999999999, 59.399999999999999]

In [11]:
date_list = []

for year in range(1928,1973):
    for month in range(1,13):
        date_list.append(dt.date(year,month,1))
        
date_list[:5]

[datetime.date(1928, 1, 1),
 datetime.date(1928, 2, 1),
 datetime.date(1928, 3, 1),
 datetime.date(1928, 4, 1),
 datetime.date(1928, 5, 1)]

In [12]:
temp_series = pd.Series(temp_list, index=date_list, name='temp')
temp_series.head()

1928-01-01    34.0
1928-02-01    33.0
1928-03-01    38.8
1928-04-01    48.4
1928-05-01    59.4
Name: temp, dtype: float64

In [13]:
# save the future (1971 - 1972)
temp_future = temp_series[temp_series.index>=dt.date(1971,1,1)]
temp_series = temp_series[temp_series.index<dt.date(1971,1,1)]
temp_series.tail()

1970-08-01    77.6
1970-09-01    70.8
1970-10-01    58.9
1970-11-01    48.5
1970-12-01    34.4
Name: temp, dtype: float64

#### Monthly avg precipitation data

In [15]:
# monthly precipitation data
# don't run this over and over again - will run out of API credits
#pdfToTable('centralpark_monthlyannualprecip.pdf', 'fehzjlvcj5ky', 'csv', 'centralpark_monthlyannualprecip.csv')
precip = pd.read_csv('centralpark_monthlyannualprecip.csv', header=2)
precip.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL
0,1869,2.53,6.87,4.61,1.39,4.15,4.40,3.20,1.76,2.81,6.48,2.03,5.02,45.25
1,1870,4.41,2.83,3.33,5.11,1.83,2.82,3.76,3.07,2.52,4.97,2.42,2.18,39.25
2,1871,2.07,2.72,5.54,3.03,4.04,7.05,5.57,5.60,2.34,7.50,3.56,2.24,51.26
3,1872,1.88,1.29,3.74,2.29,2.68,2.93,7.83,6.29,2.95,3.35,4.08,3.18,42.49
4,1873,5.34,3.80,2.09,4.16,3.69,1.28,4.61,9.56,3.14,2.73,4.63,2.96,47.99


In [16]:
# clean up:
# remove rows with repeated column names from original PDF
precip = precip[precip['YEAR']!='YEAR'] 

# convert YEAR to int
precip['YEAR'] = precip['YEAR'].astype(int) 

# keep relevant years
precip = precip[(precip.YEAR>=1928) & (precip.YEAR<=1972)] 

# convert month values to floats
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
          'JUL', 'AUG', 'SEP','OCT', 'NOV', 'DEC']

precip[months] = precip[months].astype(float)

# drop ANNUAL column
precip.drop('ANNUAL', axis=1, inplace=True)

precip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 60 to 105
Data columns (total 13 columns):
YEAR    45 non-null int64
JAN     45 non-null float64
FEB     45 non-null float64
MAR     45 non-null float64
APR     45 non-null float64
MAY     45 non-null float64
JUN     45 non-null float64
JUL     45 non-null float64
AUG     45 non-null float64
SEP     45 non-null float64
OCT     45 non-null float64
NOV     45 non-null float64
DEC     45 non-null float64
dtypes: float64(12), int64(1)
memory usage: 4.9 KB


In [17]:
# reshape into a series
precip_list = []

for _, row in precip.iterrows():
    for month in range(1,13):
        precip_list.append(row[month])
    
precip_list[:5]

[1.8700000000000001,
 4.8399999999999999,
 2.8300000000000001,
 4.8300000000000001,
 2.0699999999999998]

In [18]:
precip_series = pd.Series(precip_list, index=date_list, name='precip')
precip_series.head()

1928-01-01    1.87
1928-02-01    4.84
1928-03-01    2.83
1928-04-01    4.83
1928-05-01    2.07
Name: precip, dtype: float64

In [19]:
# save the future (1971 - 1972)
precip_future = precip_series[precip_series.index>=dt.date(1971,1,1)]
precip_series = precip_series[precip_series.index<dt.date(1971,1,1)]
precip_series.tail()

1970-08-01    2.47
1970-09-01    1.74
1970-10-01    2.48
1970-11-01    5.14
1970-12-01    2.82
Name: precip, dtype: float64

## Export the series

In [21]:
temp_series.to_csv('temp_series.csv')
precip_series.to_csv('precip_series.csv')